In [6]:
import numpy as np
import pandas as pd
import os

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.neural_network import BernoulliRBM

folder = os.getcwd() ; print folder

/home/arda/Documents/DSG


In [7]:
#pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_tagged')
#pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_tagged')

pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_munged')
pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_munged')

In [27]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key].values



list_pattern = ['\\w{1,}','(?u)\\b\\w+\\b','\w+|[,.?!;]','\w+|[$,_.?!;]']
#stop_words=['a','you','he','it','him','?','all','were', 'or','can','good']
stop_words=None


count = CountVectorizer(lowercase=False, stop_words=stop_words, token_pattern=list_pattern[2],
                        ngram_range=(1, 1), analyzer=u'word',max_df=1., min_df=2, 
                        max_features=None, vocabulary=None, binary=True)

tfidf= TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

transormData = Pipeline([
        ('selector', ColumnSelector(key='Sentence')),
        ('count', count),
#        ('tfidf', tfidf),
        #('StdScl', StdScl),
    ])

Y = pd_train['Author']
X = transormData.fit_transform(pd_train,Y) ; print X.shape
X_test = transormData.transform(pd_test)

(28723, 13328)


In [10]:
clf = BernoulliRBM(n_components=100, learning_rate=0.1, batch_size=10, n_iter=10, verbose=0, random_state=None)

In [11]:
clf.fit(X)

BernoulliRBM(batch_size=10, learning_rate=0.1, n_components=100, n_iter=10,
       random_state=None, verbose=0)

In [12]:
X_new = clf.transform(X)

In [25]:
X_new.shape

(28723, 100)